In [1]:
import pandas as pd
import numpy as np
import requests
import time

from bs4 import BeautifulSoup

from tqdm.notebook import tqdm 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_path = "C:\Program Files\Google\Chrome\Application\chrome.exe"  # 실제 경로
chrome_options.binary_location = chrome_path

driver_path = "chromedriver.exe"
myservice = Service(driver_path)
driver = webdriver.Chrome(service=myservice, options=chrome_options)

print(type(driver))

wait_time = 10
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [3]:
driver.maximize_window()

In [4]:
# 제목 / 작가 / 평점
weekdays = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
base_url = "https://comic.naver.com/webtoon?tab="
weekly_webtoons = {}

for weekday in weekdays:
    page_url = f"{base_url}{weekday}"
    driver.get(page_url)
    time.sleep(15)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    title_tags = soup.find_all("span", class_="ContentTitle__title--e3qXt")
    author_tags = soup.find_all("a", class_="ContentAuthor__author--CTAAP")
    star_tags = soup.find_all("span", class_="Rating__star_area--dFzsb") 
    img_tags = soup.find_all("img", class_="Poster__image--d9XTI")
    link_tags = soup.select("a.Poster__link--sopnC")
    
    webtoon_data = []
    for idx in range(3, len(title_tags)):
        title = title_tags[idx].get_text(strip=True)
        author = author_tags[idx].get_text(strip=True) if idx < len(author_tags) else ""
        star = star_tags[idx].get_text(strip=True)[2:] if idx < len(star_tags) else ""

        # 이미지
        img = img_tags[idx]['src'] if idx < len(img_tags) else "" 
        # 링크
        link =  "https://comic.naver.com"+ link_tags[idx]['href'] if idx < len(link_tags) else ""  
        
        webtoon_data.append([title, author, star, img, link])
        
    weekly_webtoons[weekday] = webtoon_data
    print(f"{weekday} 요일 {len(webtoon_data)}개 수집 완료\n")

# driver.quit()

mon 요일 117개 수집 완료

tue 요일 117개 수집 완료

wed 요일 111개 수집 완료

thu 요일 132개 수집 완료

fri 요일 122개 수집 완료

sat 요일 113개 수집 완료

sun 요일 127개 수집 완료



In [7]:
# 영어 → 한글 매핑
day = {'mon': '월', 'tue': '화', 'wed': '수', 'thu': '목', 'fri': '금', 'sat': '토', 'sun': '일'}

df_webtoon = pd.DataFrame([
    (day[eng_day], title, author, star, img, link)
    for eng_day, webtoons in weekly_webtoons.items()
    for title, author, star, img, link in webtoons  # link 포함으로 수정
], columns=['요일', '제목', '작가', '평점', '이미지URL', '웹툰링크'])

df_webtoon.to_csv("weekly_webtoons.csv", index=False, encoding='utf-8-sig')
print("CSV 저장 완료")

df_loaded = pd.read_csv("file/weekly_webtoons.csv")
print(df_loaded.head(5))

CSV 저장 완료
  요일           제목             작가    평점  \
0  월      만남어플 중독            루즌아  9.84   
1  월          참교육      채용택 / 한가람  9.89   
2  월         환생천마  JP / 부겸 / 장영훈  9.95   
3  월       왕과의 야행             보리  9.98   
4  월  마음의소리(였던 것)             조석  9.95   

                                              이미지URL  \
0  https://image-comic.pstatic.net/webtoon/818528...   
1  https://image-comic.pstatic.net/webtoon/818528...   
2  https://image-comic.pstatic.net/webtoon/817365...   
3  https://image-comic.pstatic.net/webtoon/817365...   
4  https://image-comic.pstatic.net/webtoon/817365...   

                                                웹툰링크  
0  https://comic.naver.com/webtoon/list?titleId=8...  
1  https://comic.naver.com/webtoon/list?titleId=7...  
2  https://comic.naver.com/webtoon/list?titleId=8...  
3  https://comic.naver.com/webtoon/list?titleId=8...  
4  https://comic.naver.com/webtoon/list?titleId=8...  


In [8]:
# 첫 번째 요일 (예: 'mon')의 첫 번째 웹툰만 출력
first_day = list(weekly_webtoons.keys())[0]   # 예: 'mon'
first_webtoon = weekly_webtoons[first_day][0] # 첫 번째 웹툰 데이터

print(f"요일: {first_day}")
print(f"제목: {first_webtoon[0]}")
print(f"작가: {first_webtoon[1]}")
print(f"평점: {first_webtoon[2]}")
print(f"이미지 URL: {first_webtoon[3]}")
print(f"웹툰 링크: {first_webtoon[4]}")

요일: mon
제목: 만남어플 중독
작가: 루즌아
평점: 9.84
이미지 URL: https://image-comic.pstatic.net/webtoon/818528/thumbnail/titledescimage/objectImage_52163270-ea97-47d6-bf87-35db4dc7965f.png
웹툰 링크: https://comic.naver.com/webtoon/list?titleId=839004&tab=mon
